### **Transactions & Concurrency Control**

## **Q1) What are ACID properties in databases, and why are they important for ensuring consistency?**  
### **Answer:**  
ACID stands for **Atomicity, Consistency, Isolation, and Durability**. These properties ensure reliable transaction processing in databases.

1. **Atomicity**  
   - Ensures that a transaction is treated as a single unit, either **fully completed** or **fully rolled back** if an error occurs.  
   - Example:  
     ```sql
     BEGIN TRANSACTION;
     UPDATE accounts SET balance = balance - 500 WHERE account_id = 1;
     UPDATE accounts SET balance = balance + 500 WHERE account_id = 2;
     COMMIT;
     ```
     If the second update fails, the first update is rolled back.

2. **Consistency**  
   - Ensures that transactions move the database from one valid state to another. Constraints like **foreign keys, unique constraints, and triggers** help maintain consistency.  
   - Example: If an order is placed, the stock quantity must be reduced accordingly.

3. **Isolation**  
   - Prevents transactions from interfering with each other. Isolation levels (Read Uncommitted, Read Committed, Repeatable Read, Serializable) define how transactions interact.  
   - Example: A transaction modifying a row should not be visible to others until committed.

4. **Durability**  
   - Ensures that once a transaction is committed, it is permanently saved, even in case of system failure.  
   - Example: Writing transaction logs to disk ensures data persistence.

## **Q2) What are the different types of transaction isolation levels? How do they affect concurrency and performance?**  
### **Answer:**  
SQL databases support **four isolation levels** that control the visibility of uncommitted changes to other transactions:

1. **Read Uncommitted**  
   - Transactions can read uncommitted changes (dirty reads).  
   - **Issue:** Dirty reads, non-repeatable reads, and phantom reads can occur.  
   - **Performance:** High concurrency, but low consistency.  
   - **Example:**
     ```sql
     SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED;
     ```

2. **Read Committed** (Default in most databases)  
   - A transaction only reads committed data.  
   - **Issue:** Prevents dirty reads but allows non-repeatable reads.  
   - **Performance:** Moderate concurrency and consistency.  
   - **Example:**
     ```sql
     SET TRANSACTION ISOLATION LEVEL READ COMMITTED;
     ```

3. **Repeatable Read**  
   - Ensures the same row will not change within a transaction.  
   - **Issue:** Prevents dirty and non-repeatable reads but allows phantom reads.  
   - **Performance:** Lower concurrency than Read Committed.  
   - **Example:**
     ```sql
     SET TRANSACTION ISOLATION LEVEL REPEATABLE READ;
     ```

4. **Serializable**  
   - Transactions are fully isolated.  
   - **Issue:** Prevents all anomalies but is the slowest due to strict locking.  
   - **Performance:** Lowest concurrency, highest consistency.  
   - **Example:**
     ```sql
     SET TRANSACTION ISOLATION LEVEL SERIALIZABLE;
     ```

## **Q3) Explain the difference between pessimistic and optimistic locking. When would you use each?**  
### **Answer:**  

| Locking Type | How It Works | Use Case |
|-------------|-------------|----------|
| **Pessimistic Locking** | Prevents other transactions from modifying a resource until the lock is released. | Used when conflicts are **frequent** (e.g., banking transactions). |
| **Optimistic Locking** | Allows multiple transactions to read the same data but checks for changes before committing. | Used when conflicts are **rare** (e.g., online form submissions). |

### **Example of Pessimistic Locking (SQL Server)**
```sql
BEGIN TRANSACTION;
SELECT * FROM accounts WITH (XLOCK) WHERE account_id = 1;
UPDATE accounts SET balance = balance - 500 WHERE account_id = 1;
COMMIT;
```
- The `XLOCK` ensures no other transaction can access the row.

### **Example of Optimistic Locking**
```sql
UPDATE accounts 
SET balance = balance - 500 
WHERE account_id = 1 AND last_updated = '2024-02-04 10:00:00';
```
- The `WHERE` clause ensures that if another transaction modifies the data, this update fails.

---

## **Q4) How do deadlocks occur in a database, and how would you prevent or resolve them?**  
### **Answer:**  

### **What is a Deadlock?**  
A **deadlock** occurs when two transactions hold locks that each other needs, preventing both from proceeding.

### **Example of a Deadlock**
```sql
-- Transaction 1
BEGIN TRANSACTION;
UPDATE accounts SET balance = balance - 500 WHERE account_id = 1;
UPDATE accounts SET balance = balance + 500 WHERE account_id = 2;

-- Transaction 2
BEGIN TRANSACTION;
UPDATE accounts SET balance = balance - 500 WHERE account_id = 2;
UPDATE accounts SET balance = balance + 500 WHERE account_id = 1;
```

### **How to Prevent Deadlocks?**
1. **Ensure a consistent locking order** (Always lock tables in the same order).
2. **Use shorter transactions** (Commit as soon as possible).
3. **Use lower isolation levels** (e.g., Read Committed instead of Serializable).
4. **Enable deadlock detection** (Most databases detect and resolve deadlocks by rolling back one transaction).

---

## **Q5) What is MVCC (Multi-Version Concurrency Control), and how does it work in databases like PostgreSQL?**  
### **Answer:**  
**MVCC (Multi-Version Concurrency Control)** allows multiple versions of a record to exist simultaneously, ensuring **high concurrency**.

### **How it Works**
1. **Read transactions see a snapshot** of the database at the start of their execution.
2. **Writes create a new version** of the row instead of modifying the existing one.
3. **Older versions are cleaned up** by a background process.

### **Example**
```sql
BEGIN TRANSACTION;
SELECT * FROM accounts WHERE account_id = 1;
-- Another transaction updates account_id = 1, but this transaction still sees the old value.
COMMIT;
```

- PostgreSQL and MySQL (InnoDB) use MVCC for **better performance than locks**.

---

## **Q6) What is a transaction in SQL, and why is it important?**  
### **Answer:**  
A **transaction** is a sequence of SQL operations that are executed as a single unit, ensuring **data integrity**.

### **Example**
```sql
BEGIN TRANSACTION;
UPDATE accounts SET balance = balance - 500 WHERE account_id = 1;
UPDATE accounts SET balance = balance + 500 WHERE account_id = 2;
COMMIT;
```
- Transactions **guarantee atomicity** (all steps succeed or fail together).

---

## **Q7) Explain the ACID properties (Atomicity, Consistency, Isolation, Durability).**  
- Already answered in **Q1**.

---

## **Q8) What is transaction isolation, and what are the different levels?**  
- Already answered in **Q2**.

---

## **Q9) What are common issues related to concurrent transactions, and how would you address them?**  
### **Answer:**  

| Issue | Description | Solution |
|------|------------|----------|
| **Dirty Reads** | Reading uncommitted data. | Use **Read Committed** or higher. |
| **Non-Repeatable Reads** | A row changes between two reads. | Use **Repeatable Read** or higher. |
| **Phantom Reads** | A new row appears in a repeated query. | Use **Serializable**. |
| **Lost Updates** | Two transactions overwrite each other's changes. | Use **Optimistic Locking**. |
| **Deadlocks** | Two transactions block each other. | Follow a **consistent locking order**, use **timeouts**, or use **deadlock detection**. |

---

These detailed explanations should help you understand database transactions, concurrency, and locking mechanisms comprehensively. Let me know if you need further clarification! 🚀